In [1]:
import pandas as pd
import numpy as np

In [2]:
import spacy
import en_core_web_sm
svoc = spacy.load("en_core_web_sm")

In [3]:
tokenizertrg = lambda x: x.split()
def tokenizersrc(text): # create a tokenizer function
    return [tok.text for tok in svoc.tokenizer(text)]

In [4]:
datao = pd.read_pickle("~/OneDrive/kph/processed3.pkl")

In [5]:
datao = datao.replace(r'\W+',' ', regex=True)

In [6]:
datao['abs_l'] = (datao['abskey'].apply(lambda x: len(x)))

In [14]:
datao['title'] = 'a'

In [8]:
datao['keyword'] = (datao['abskey'].apply(lambda x: ';'.join(x)))

In [7]:
datau = datao[datao['Extracted']>=3]

In [8]:
datat = datao[datao['Extracted']<3]

In [47]:
datao

,SRC,TRG,keywords,Extracted,abskey
32460418,Chromatin remodelers in oligodendroglia. Oligo...,B I O O O O O O O O O O O O O O O O O O O O O ...,"[CNS development, chromatin remodelers, oligod...",1,"[CNS development, oligodendrogenesis, transcri..."
32460167,Clinical implications of serum neurofilament i...,O O O O O O O O O O O O O O O O O O O O O O B ...,"[Biomarker, Multiple sclerosis, Neurofilament ...",4,[Prediction]
32460145,Effect of cognitive rehabilitation on neuropsy...,O O B I O O O O O O O O O O O B I O O O O O O ...,"[Clinical trial, Cognition, Cognitive rehabili...",4,"[Clinical trial, Cognition]"
32460086,B-cell depleting therapies may affect suscepti...,B I I I I O O O O O O O O O O B I O O O B O O ...,"[B-cell depleting therapies, COVID-19, DMTs, M...",4,[]
32459513,The dichotomous role of the gut microbiome in ...,O O O O O B I O O O O B I O O O B I O O O O O ...,"[Alzheimer’s disease, Parkinson’s disease, gut...",5,"[Alzheimer’s disease, Parkinson’s disease, gut..."
...,...,...,...,...,...
13106716,Multiple sclerosis; its frequency and distribu...,O O O O O O O O O O O O O O O O O O O O O O O ...,[MULTIPLE SCLEROSIS/epidemiology],0,[MULTIPLE SCLEROSIS/epidemiology]
13106715,The allergic aspects of multiple sclerosis. Fo...,O O O O O O O O O O O O O O O O O O O O O O O ...,[ALLERGY/manifestations],0,[ALLERGY/manifestations]
13106714,Histamine treatment of multiple sclerosis. His...,O O O O O O O O O O O O O O O O O O O O O O O ...,"[HISTAMINE/therapeutic use, MULTIPLE SCLEROSIS...",0,"[HISTAMINE/therapeutic use, MULTIPLE SCLEROSIS..."
14832406,Studies on acute disseminated encephalomyeliti...,O O O O B O O O O O O O O O O O O O O O O O O ...,"[CEREBROSPINAL FLUID, ENCEPHALOMYELITIS]",2,[]


In [50]:
# separate train and validate 
VAL_RATIO = 0.2
dtrain = datau.loc[:,['SRC','TRG']]
dtraink = datau.loc[:,['SRC','TRG']]
seed=250
idx = np.arange(datau.shape[0])
np.random.seed(seed)
np.random.shuffle(idx)
val_size = int(len(idx) * VAL_RATIO)
df_train = dtrain.iloc[idx[val_size:], :]
df_val = dtrain.iloc[idx[:val_size], :]

In [15]:
datao = datao.rename(columns={'SRC':'abstract'})
datau = datao[datao['abs_l']>=2]
datat = datao[(datao['abs_l']<2) & (datao['abs_l']>0)]
# separate train and validate 
VAL_RATIO = 0.2
dtrain = datau.loc[:,['abstract','keyword','title']]
dtraink = datau.loc[:,['abstract','keyword','title']]
seed=250
idx = np.arange(datau.shape[0])
np.random.seed(seed)
np.random.shuffle(idx)
val_size = int(len(idx) * VAL_RATIO)
df_train = dtrain.iloc[idx[val_size:], :]
df_val = dtrain.iloc[idx[:val_size], :]
df_test = datat.loc[:,['abstract','keyword','title']]

In [19]:
df_val

,abstract,keyword,title
31680631,Diroximel fumarate DRF in patients with relaps...,Relapsing–remitting multiple sclerosis;clinica...,a
28285265,Neuropsychiatric syndromes of multiple scleros...,Bipolar Disorder;Depression;Magnetic Resonance...,a
28203098,Health care resource utilization before and af...,outcomes;relapsing–remitting,a
28607730,Distractibility in multiple sclerosis The role...,cognition;distraction,a
32172998,The multiple sclerosis gut microbiota A system...,Bacteria;Microbiome,a
...,...,...,...
29299614,Cortical and Subcortical Morphometric and Iron...,Cortical grey matter;Magnetic susceptibility m...,a
27424141,Mannose binding Lectin Mediated Complement Pat...,Autoimmunity;Complement system proteins,a
19966907,Coenzyme Q10 effects in neurodegenerative dise...,Huntington’s disease;Parkinson’s disease,a
31443863,Using centrality measures to extract core patt...,Coarse-graining;Discrete dynamics;Entropy rate...,a


In [20]:
df_train.to_json('kp20k_training.json',orient='records',lines=True)

In [21]:
df_val.to_json('kp20k_validation.json',orient='records',lines=True)

In [22]:
df_test.to_json('kp20k_testing.json',orient='records',lines=True)

In [21]:
df_train.loc[:,'abskey'] = df_train.loc[:,'abskey'].replace(',',';', regex=True)

/home/pding/anaconda3/envs/kph2/lib/python3.8/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [26]:
df_train['abskey'] = df_train['abskey'].str.replace(',',';')

<ipython-input-26-2d130bf99ece>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['abskey'] = df_train['abskey'].str.replace(',',';')


In [33]:
df_train.abskey[0]

['CanMEDS', 'Health advocate', 'education']

In [40]:
t1 = dataot.iloc[1,0]

In [53]:
file1 = open("trainan.txt","w")

In [54]:
for _, row in df_train.iterrows():
    text = tokenizersrc(row[0])
    label = tokenizertrg(row[1])
    for i in range(len(text)):
        file1.write(r'%s' %text[i])
        file1.write('###')
        file1.write(r'%s' %label[i])
        file1.write(' ')
    file1.write('\n')

In [55]:
file1.close() 

In [56]:
file2 = open("testan.txt","w")
for _, row in df_val.iterrows():
    text = tokenizersrc(row[0])
    label = tokenizertrg(row[1])
    for i in range(len(text)):
        file2.write(r'%s' %text[i])
        file2.write('###')
        file2.write(r'%s' %label[i])
        file2.write(' ')
    file2.write('\n')
file2.close() 

In [31]:
print(r'%s' % a)

tt 
 tt


In [37]:
myVariable = 'This string is supposed to be raw \n '
print(r'%s' %myVariable)

This string is supposed to be raw 
 
